# Problem Set 2




# Problem 1


In [1]:
from scipy.linalg import toeplitz
import numpy as np
import math
import scipy.io.wavfile as wav
import matplotlib.pyplot as plt
from IPython.display import Audio
from matplotlib import rcParams as rc

Возьмите такую часть сигнала, чтобы матрица $N \times N$ и операции над ней не вызывали свапа. Ответ обосновать, расчитать около максимальное такое $N$

Сигнал нужно теперь свернуть со следующей матрицей:
$$y_{convolved} = T x$$
$$T_{ij} = \sqrt{\frac{\alpha}{\pi}}e^{-\alpha (i-j)^2}, \quad i,j=1,\dots,N$$

- Реализуйте функцию, которая создает такую матрицу, не используя списки и циклы, как называются такие матрицы ?
- Реализуйте функцию, которая возвращает свертку
- Изобразите для $\alpha = 0.2,~\alpha=0.01$ на одном графике исходный сигнал и свернутый (два графика соответственно), изобразите только часть данных для репрезентативности.
- Воспроизведите свернутый сигнал в виде аудио

А теперь давайте попробуем его восстановить. Выражение для получения свернутого сигнала есть ни что иное как СЛАУ с неизвестным исходным сигналом. Давайте решим это уравнение итерационным методом.

- Реализуйте метод Ричарсона с оптимальным шагом
- Реализуйте метод Ричарсона с переменным шагом, который на каждой итерации минимизирует норму невязки, выражение для оптимального шага получить аналитически
- Реализуйте метод Чебышева (с перестановкой корней задание со звездочкой)
- Для каждого $\alpha$ постройте график относительной погрешности реализованных методов.
- Добавьте небольшой случайный шум к свернутому сигналу ($10^{-3}\max(x)$), постройте графики сходимости, обоснуйте результаты на них.



In [ ]:
###

from scipy.linalg import toeplitz
import numpy as np
import math
import scipy.io.wavfile as wav
import matplotlib.pyplot as plt
from IPython.display import Audio
from matplotlib import rcParams as rc
from scipy.linalg import qr
import wave
import sys
import numpy as np
from scipy import fft, ifft
from scipy.fftpack import fftfreq
import scipy as sp
import scipy.sparse
import scipy.sparse.linalg as spla
import scipy
import  struct, random
def creatMatrix(N,alpha):
	d = np.zeros((N,N))
	for i in range(0,N):
		k = math.sqrt(alpha/3.14) * math.exp(-alpha*((i)^2))
		b = k * np.eye(N, N, i)
		d = d + b
	Q, R = qr(d)
	c = Q.dot(R.T)
	FinMatrix = d + c - np.eye(N,N,0)
	return FinMatrix
# Нахождение свертки
def Svert(B,X):
	Y = B.dot(X)
	return Y
# Write new signal into a file
def WriteWav(Y,fs_rate,N):
	sampleRate = 44100.0 # hertz
	duration = 1.0 # seconds
	frequency = 440.0 # hertz
	obj = wave.open('sound.wav','w')
	obj.setnchannels(1) # mono
	obj.setsampwidth(2)
	obj.setframerate(fs_rate)
	for i in range(N):
	   #value = Y[i]
	   #data = struct.pack('<h', value)
	   obj.writeframesraw( Y[i] )
	obj.close()
def ReadWav(FileName):
	fs_rate, signal = wav.read(FileName)
	l_audio = len(signal.shape)
	if l_audio == 2:
		signal = signal.sum(axis=1)/2
	N = signal.shape[0]
	secs = N / float(fs_rate)
	Ts = 1.0/fs_rate # sampling interval in time
	t = np.arange(0, secs, Ts)
	if N > 9999:
		NewSignal = signal[0:1000]
		N = 1000
	else:
		NewSignal = signal 
	return t,N,fs_rate,signal,NewSignal


def GenMat(N,alpha):
	k = math.sqrt(alpha/3.14)
	As = np.ones((2*N-1,2*N-1))
	Xr = np.ones(N+1)
	Place = np.arange(-N+1,N,1)
	arr1 = np.arange(0,N,1)
	arr2 = np.arange(N-1,0,-1)
	Pow = np.concatenate((arr2, arr1))
	data = np.exp(-alpha*Pow)
	Mat = As*data
	Mat = Mat.T
	A = spdiags(Mat, Place, N, N,'csr')
	return A


import matplotlib.pyplot as plt
def PlOne(signal,t):
	p1 = plt.plot(t, signal, "g") # plotting the signal
	plt.xlabel('Time')
	plt.ylabel('Amplitude')
	plt.show()
def PlTwo(N,signal,Y,t):
	tnew = t[0:N]
	signalcut = signal[0:N]
	p1 = plt.plot(tnew, signalcut, "g")
	p2 = plt.plot(tnew,Y,"r")
	plt.xlabel('Time')
	plt.ylabel('Amplitude')
	plt.legend(["Original", "Svert"])
	plt.show()
    
    
    def Rich(A,b,n):
	rhs = b
	ev1, vec = spla.eigsh(A, k=2, which='LA')
	ev2, vec = spla.eigsh(A, k=2, which='SA')
	lam_max = ev1[0]
	lam_min = ev2[0]
	tau_opt = 2.0/(lam_max + lam_min)
	niters = 100
	x = np.zeros(n)
	res_richardson = []
	for i in range(niters):
	    rr = A.dot(x) - rhs
	    x = x - tau_opt * rr
	    res_richardson.append(np.linalg.norm(rr))
	#Convergence of an ordinary Richardson (with optimal parameter)
	#print(res_richardson)
	plt.plot(res_richardson)
	plt.yscale('log')
	plt.xlabel("Number of iterations", fontsize=20)
	plt.ylabel("Residual norm", fontsize=20)
	plt.xticks(fontsize=20)
	plt.yticks(fontsize=20)
	print("Maximum eigenvalue = {}, minimum eigenvalue = {}".format(lam_max, lam_min))
	print("Condition number = {}".format(lam_max.real / lam_min.real))
	return x


###main


FileName = "1.wav"
t,N,fs_rate,signal,NewSignal = ReadWav(FileName)
alpha = 0.2
Mat = GenMat(N,alpha)
print(Mat)
Y = Svert(Mat,NewSignal)
PlOne(signal,t)
PlTwo(N,signal,Y,t)
WriteWav(Y,fs_rate,N)
X = Rich(Mat,Y,N)

# Problem 3

Решиьт интегральное уравнение (ИУ) Фредгольма второго рода
$$u(x) = \lambda \int\limits_{-\pi}^{\pi}|x - y|u(y)dy + f(x),$$
где $|x - y| = K(x, y)$ -- ядро ИУ, $f(x) = (1 + 2\lambda)\cos^2\frac{x}{2} - \lambda\frac{x^2 + \pi^2}{2}$

Уравнение на сетке может быть сведено к следующей системе линейных уравнений:

$$\sum_{m = 1}^N[\delta_{nm} - \lambda K_{nm}]u_m = f_n$$

$$K_{nm} = h^2|n - m| \in \mathbb{R}^{N \times N}$$
$$f_n = f(nh - h/2 - \pi)$$

Точное решение $u(x) = \cos^2\frac{x}{2}$, ошибка метода $O(h^2)$, $\lambda = 0.01$

Решить систему используя:
- Якоби
- Методом Зейделя
- Покажите, что метод Зейделя может быть  рассмотрен как покоординатный спуск
- SOR(Successive overrelaxation), исследовать поведения метода в зависимости от параметра релаксации

Для всех методов выше привести графики сходимости.

- Какая сложность методов для плотных и разреженных матриц?

In [ ]:
###SOR

import numpy as np
def sor_solver(A, b, omega, initial_guess, convergence_criteria):
  phi = initial_guess[:]
  residual = np.linalg.norm(np.matmul(A, phi) - b) #Initial residual
  while residual > convergence_criteria:
    for i in range(A.shape[0]):
      sigma = 0
      for j in range(A.shape[1]):
        if j != i:
          sigma += A[i][j] * phi[j]
      phi[i] = (1 - omega) * phi[i] + (omega / A[i][i]) * (b[i] - sigma)
    residual = np.linalg.norm(np.matmul(A, phi) - b)
#    print('Residual: {0:10.6g}'.format(residual))
  return phi

###Zeydel

import numpy as np
def sel(A,b):
  ITERATION_LIMIT = 1000
  x = np.zeros_like(b)
  for it_count in range(1, ITERATION_LIMIT):
      x_new = np.zeros_like(x)
      for i in range(A.shape[0]):
          s1 = np.dot(A[i, :i], x_new[:i])
          s2 = np.dot(A[i, i + 1 :], x[i + 1 :])
          x_new[i] = (b[i] - s1 - s2) / A[i, i]
      if np.allclose(x, x_new, rtol=1e-8):
          break
      x = x_new
#  print("Solution: {0}".format(x))
#  error = np.dot(A, x) - b
#  print("error:",error)
  return x

###jak


import numpy as np
from scipy.linalg import solve
import math as math
import matplotlib.pyplot as plt
def jacobi(A, b, x, n):
    D = np.diag(A)
    R = A - np.diagflat(D)
    for i in range(n):
        x = (b - np.dot(R,x))/ D
    return x



###main

import numpy as np
from scipy.linalg import solve
import math as math
import matplotlib.pyplot as plt
from Jak import jacobi
from Sel import sel
from SOR import sor_solver
def RealSoulution(N):# creation of real soultuion values
	h = 2*math.pi/N
	Y = np.zeros(N)
	for i in range(0,N):
		x = (h*i-h/2-math.pi)
		Y[i] = math.cos(x/2)**2
	return Y
def GenOfInitMatrix(N,lamda): # creation of initial matrix
	E = np.eye(N)
	A = np.zeros((N,N))
	b = np.zeros(N)
	h = 2*math.pi/N
	for i in range(0,N):
		for j in range(0,N):
			K = h*h*abs(i-j)
			x = (h*i-h/2-math.pi)
			b[i] = (1+2*lamda)*math.cos(x/2)**2 - lamda*(x**2+math.pi**2)/2
			A[i][j] = E[i][j] - lamda*K
	return A,b
lamda =  0.01
N = 100 # Number of matrix
xRealSol = RealSoulution(N)
A,b = GenOfInitMatrix(N,lamda)
n = 100 # noumber of iterations
# jakobi soulution start
y = np.ones(N)
xJak = jacobi(A, b, y, n)
# SelSuol
xSel = sel(A,b)
# SorSoul
residual_convergence = 1e-8
omega = 0.5
xSor = sor_solver(A, b, omega, y, residual_convergence)
#np.savetxt("Real.txt", xSol, fmt="%s")
#np.savetxt("Jakob.txt", x, fmt="%s")
print("xJak=",xJak)
print("xSel=",xSel)
print("xSor=",xSor)
